In [ ]:
import requests
import json
import time
from datetime import datetime
import warnings
warnings.simplefilter('ignore')
from bs4 import BeautifulSoup
from IPython.display import Audio

### Variable à remplir vous-même
request_id = "b747c9fa-60ee-4b93-9740-a160388af654"
range_wanted = ("16:30", "17:30")
###

### Modifiable si vous le souhaitez, mais ce serait sympa d'éviter de les ddos
delay_between_tries = 60
nb_tries = 100
###


def str_to_time(t_str):
    return datetime.strptime(t_str, "%H:%M").time()


def play_alert_sound():
    return Audio("alert.mp3", autoplay=True)


# On peut récupérer une clé en cours de validité sur la page d'accueil de sncf-connect. Je pense que cette clé doit changer régulièrement ?
def get_api_key():
    headers = {
        "Host": "www.sncf-connect.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    response = requests.get("https://www.sncf-connect.com", verify=False, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/json'})
        data = json.loads(script_tag.string)
        x_bff_key = data['bffServer']['key_value']
        return x_bff_key

    else:
        print(f"Erreur : {response.status_code}")
        raise Exception("Erreur dans la récupération de la clé API")


# La SNCF utilise le service anti-bot datadome. Il faut obtenir un cookie datadome pour pouvoir taper l'API. Normalement on l'obtient en résolvant un captcha mais est-ce que
# c'est bien fait ? visiblement non
def get_datadome():
    url = "https://www.sncf-connect.com/bff/api/v1/itineraries"
    response = requests.get(url)
    cookies = response.cookies
    cookie_dict = requests.utils.dict_from_cookiejar(cookies)
    return cookie_dict['datadome']
    

start_time = str_to_time(range_wanted[0])
end_time = str_to_time(range_wanted[1])
i=0
while(i < nb_tries):
    i += 1
    print(f"## Tentative de {datetime.now()}")
    print()
    
    headers = {
        "Host": "www.sncf-connect.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "x-bff-key": get_api_key(),
        "Connection": "keep-alive",
        "Referer": "https://www.sncf-connect.com/app/home/shop/results/outward",
    }
    
    cookies = {
        "datadome": get_datadome(),
    }
    
    # on récupère les trajets proposés et le statut de chaque trajet
    response = requests.get(f"https://www.sncf-connect.com/bff/api/v1/itineraries/{request_id}?outward=true&plan=false", verify=False, headers=headers, cookies=cookies)

    if response.status_code == 200:
        results = response.json()
        proposals = results.get('output', {}).get('longDistance', {}).get('proposals', {}).get('proposals', [])
        
        found_match = False
        for proposal in proposals:
            dep_time_str = proposal['departure']['timeLabel']  # e.g. "16:45"
            dep_time = str_to_time(dep_time_str)
            is_bookable = proposal['status']['isBookable']

            print(f"{dep_time_str} : Bookable={is_bookable}")

            # Check if departure time is within the wanted range and bookable
            if start_time <= dep_time <= end_time and is_bookable:
                print(f"*** Found a bookable trip in the desired time range: {dep_time_str} ***")
                found_match = True
                play_alert_sound()
                break
        if found_match:
            break
        else:
            print("Not yet ...")
    else:
        # Printing an error message if the request failed
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        
    time.sleep(delay_between_tries)

## Tentative de 2024-12-18 14:06:37.187078

15:35 : Bookable=True
15:35 : Bookable=True
16:31 : Bookable=True
*** Found a bookable trip in the desired time range: 16:31 ***
